# Step 1: Make the images background transparent

In [ ]:
from PIL import Image
import os

# Paths
source_folder = "./source_folder"
destination_folder = "./destination_folder"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Iterate over all files in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        img_path = os.path.join(source_folder, filename)
        img = Image.open(img_path).convert("RGBA")  # Ensure image is in RGBA mode
        
        # Get image data
        datas = img.getdata()
        new_data = []
        
        for item in datas:
            if item[0] == 0 and item[1] == 0 and item[2] == 0:  # Black color
                new_data.append((255, 255, 255, 0))  # Make black pixels transparent
            else:
                new_data.append(item)
        
        # Update image data with new transparent data
        img.putdata(new_data)
        
        # Save the image with a transparent background
        img.save(os.path.join(destination_folder, filename))

print("All images have been processed and saved in the destination folder.")

# Step 2: Add the background color to transparent images

In [ ]:
import cv2
import os
import numpy as np

# Define paths
input_images_path = r"./source_folder"
output_images_path = r"./destination_folder"

# Create output directory if it doesn't exist
os.makedirs(output_images_path, exist_ok=True)

# Function to create a solid grey background
def create_solid_grey_background(width, height):
    return np.full((height, width, 3), (128, 128, 128), dtype=np.uint8)

# Process each transparent image
for image_file in os.listdir(input_images_path):
    if image_file.endswith('.png'):
        img_path = os.path.join(input_images_path, image_file)
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # Separate the alpha channel if it exists
        if img.shape[2] == 4:
            alpha_channel = img[:, :, 3]
            img_rgb = img[:, :, :3]
        else:
            raise ValueError("Image does not have a transparent background")

        # Create the grey background with the same size as the image
        height, width = img_rgb.shape[:2]
        grey_background = create_solid_grey_background(width, height)

        # Create a mask from the alpha channel
        mask = alpha_channel / 255.0

        # Composite the image onto the grey background using the alpha mask
        for c in range(3):
            grey_background[:, :, c] = (mask * img_rgb[:, :, c] + (1 - mask) * grey_background[:, :, c])

        # Save the result
        output_path = os.path.join(output_images_path, image_file)
        cv2.imwrite(output_path, grey_background)

print("Images with grey background created successfully.")